In [ ]:
# Fall 2024 - MH
# Environment: fmri_stats
# Organize and plot item-level GLMER results for delta RSA neural score predicting quiz scores

In [5]:
# SET-UP
import numpy as np
import pandas as pd
import scipy.stats as st
import copy
import pickle
import matplotlib.pyplot as plt

from pymer4.models import Lmer

base_dir = '/dartfs-hpc/rc/lab/K/KraemerD/ASL1-2_combined/'
rsa_dir=base_dir+'data/rsa/'
glmer_dir = base_dir+'data/glmer_item_scores/'

subj_dict = {'001':{'group':'1'},'003':{'group':'2'},'004':{'group':'1'},
     '005':{'group':'1'},'006':{'group':'1'},'007':{'group':'1'},'008':{'group':'1'},
     '009':{'group':'1'},'010':{'group':'1'},'011':{'group':'1'},'012':{'group':'1'},
     '013':{'group':'2'},'014':{'group':'1'},'015':{'group':'2'},'016':{'group':'2'},
     '017':{'group':'2'},'018':{'group':'2'},'019':{'group':'2'},'020':{'group':'2'},
     '021':{'group':'2'},'022':{'group':'2'},'023':{'group':'2'},'024':{'group':'2'},
     '025':{'group':'2'},'026':{'group':'2'},'027':{'group':'2'},'028':{'group':'1'},
     '029':{'group':'2'},'030':{'group':'2'},'031':{'group':'1'},'032':{'group':'2'},
     '033':{'group':'1'},'034':{'group':'1'},'035':{'group':'1'},'036':{'group':'1'},
     '037':{'group':'1'},'038':{'group':'1'},'040':{'group':'1'},'041':{'group':'2'},
     '042':{'group':'2'}}

subs= subj_dict.keys()

words = [['goat','hippo','ostrich','turtle','coconut','lemon','peach','pineapple','airplane','bulldozer','bus','spaceship'],
         ['dog','hen','mouse','tiger','apple','cherry','melon','strawberry','helicopter','sailboat','train','truck']]

n_parcs = 500 # parcellation resolution

# read in behavioral quiz scores
beh = pd.read_csv(base_dir+'data/behavioral/ASL2_summary_quiz_scores_all.csv')

# significant parcels determined by parcel-selection RSA:
eng_parcs = [1, 4, 7, 19, 23, 26, 27, 32, 33, 34, 59, 60, 63, 64, 66, 67, 70, 72, 78, 87, 88, 92, 93, 94, 97, 102, 104, 114,
             118, 119, 122, 124, 126, 129, 132, 135, 137, 139, 144, 156, 159, 162, 164, 168, 170, 177, 178, 181, 198, 201, 202,
             207, 208, 209, 212, 215, 219, 227, 228, 229, 230, 233, 248, 249, 264, 272, 277, 279, 280, 301, 311, 314, 317, 326,
             339, 342, 350, 352, 356, 358, 359, 363, 368, 370, 375, 381, 386, 398, 399, 407, 414, 417, 425, 434, 445, 447, 455,
             472, 473, 474, 475, 487]

asl_parcs = [31, 47, 72, 106, 147, 200, 230, 238, 291, 304, 358, 362, 367, 382, 412, 436, 455, 457, 490]

union = list(np.unique(eng_parcs+asl_parcs))
overlap = [ x for x in eng_parcs if x in asl_parcs ]

In [9]:
### Model one parcel at a time (e.g. to create regression plots in Figure 6)

# INDEX (parcel # - 1) of parcel to plot:
parc = 370

sublist = []
grouplist = []
timepointlist = []
wordlist = []
recalllist =[]
neuralscorelist = []

for sub in subs:
    group = subj_dict[sub]['group']
    all_neuralscores = pd.read_csv(glmer_dir+'sub-'+sub+'_known_item_delta_crosslang_similarity_Schaefer500.csv')
    parc_neuralscores = all_neuralscores[str(parc)]

    word_recalls_all = pd.read_csv(base_dir+'data/behavioral/ASL2_grp'+str(group)+'_target_recall_scores.csv')
    word_recalls_sub = word_recalls_all[word_recalls_all['SID'] == int(sub)]

    for row in range(len(word_recalls_sub.index)):
        timepoint = word_recalls_sub['Time'].iloc[row]
        if timepoint == 't1' or timepoint =='t3':
            recalls_bin = list(list(word_recalls_sub.iloc[row])[2:])
        elif timepoint == 't2':
            recalls_bin = []
            for x in list(word_recalls_sub.iloc[row])[2:]:
                if x == 4: recalls_bin.append(1)
                else: recalls_bin.append(0)
        
        for item in range(len(words[int(group)-1])):
            sublist.append(sub)
            grouplist.append(group)
            timepointlist.append(timepoint)
            wordlist.append(words[int(group)-1][item])
            recalllist.append(recalls_bin[item])
            neuralscorelist.append(parc_neuralscores[item])

    
   
scores_long = pd.DataFrame({'subID':sublist,
                        'group': grouplist,
                        'time':timepointlist,
                        'item': wordlist,
                       'recall_score':recalllist,
                        'neuralscore':neuralscorelist})

scores_long.head()

,subID,group,time,item,recall_score,neuralscore
0,001,1,t1,goat,1.0,0.886759
1,001,1,t1,hippo,1.0,0.690963
2,001,1,t1,ostrich,1.0,0.000000
3,001,1,t1,turtle,1.0,1.599603
4,001,1,t1,coconut,1.0,1.465515


In [10]:
model = Lmer("recall_score ~ neuralscore + (1|item) + (1|time) + (1|subID)", data=scores_long, family="binomial")
results_df = model.fit()
results_df

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: recall_score~neuralscore+(1|item)+(1|time)+(1|subID)

Family: binomial	 Inference: parametric

Number of observations: 1440	 Groups: {'subID': 40.0, 'item': 24.0, 'time': 3.0}

Log-likelihood: -450.985 	 AIC: 911.970

Random effects:

              Name    Var    Std
subID  (Intercept)  1.819  1.349
item   (Intercept)  1.519  1.233
time   (Intercept)  0.595  0.771

No random effect correlations specified

Fixed effects:



,Estimate,2.5_ci,97.5_ci,SE,OR,OR_2.5_ci,OR_97.5_ci,Prob,Prob_2.5_ci,Prob_97.5_ci,Z-stat,P-val,Sig
(Intercept),2.516,1.354,3.678,0.593,12.383,3.875,39.574,0.925,0.795,0.975,4.245,0.000,***
neuralscore,0.464,0.145,0.783,0.163,1.591,1.156,2.188,0.614,0.536,0.686,2.854,0.004,**


In [ ]:
# Permutation test can't run in notebook, but details can be found in Study 2/scripts
# The model is re-fit 1,000 times, each time shuffling the mapping of neural score to behavioral scores.

In [3]:
sig_parcs = union

betaList=[]
zList=[]

predict_parcs = []
predict_Zs = []

for parc in sig_parcs:
    try:
        data = pickle.load(open(lmer_dir+'glmer_parcID-'+str(parc)+'_grp-all_delta_known_unknown_crosslang_item_similarity_score_prediction.pkl','rb'))
        betaList.append(data['beta_obs'])
        zList.append(data['beta_Z'])
        if data['beta_Z'] >= 1.95: 
            print("ParcID "+str(parc)+" predicts quiz scores with Z="+str(data['beta_Z']))
            predict_parcs.append(parc)
            predict_Zs.append(data['beta_Z'])
        elif data['beta_Z'] <= -1.95:
            print("ParcID "+str(parc)+" is negatively associated with quiz scores, Z="+str(data['beta_Z']))
            predict_parcs.append(parc)
            predict_Zs.append(data['beta_Z'])

    except:
        print("no file for p#"+str(parc))
        continue

ParcID 129 predicts quiz scores with Z=2.4906803439182785
ParcID 178 is negatively associated with quiz scores, Z=-2.773514131295342
ParcID 181 predicts quiz scores with Z=2.6214202890380403
ParcID 201 is negatively associated with quiz scores, Z=-2.4283416247156504
ParcID 207 predicts quiz scores with Z=2.4651304354498045
ParcID 208 predicts quiz scores with Z=2.1662982322132027
ParcID 209 predicts quiz scores with Z=2.154982601843967
ParcID 248 predicts quiz scores with Z=1.9814765900849851
ParcID 280 predicts quiz scores with Z=2.285838999977826
ParcID 362 predicts quiz scores with Z=2.1788873799553823
ParcID 363 predicts quiz scores with Z=2.0063010098359224
ParcID 370 predicts quiz scores with Z=2.9390556751583894
ParcID 399 predicts quiz scores with Z=2.4520005461476018


## Plot all these values in a heatmap on the FSAverage Surface

In [36]:
# Plot all these values in a heatmap on the FSAverage Surface

%run '/dartfs-hpc/rc/lab/K/KraemerD/ASL1-2_combined/scripts/asl_combo_functions_surface_plotting.py'

parc_vals = [0]*500

for p in range(len(union)):
    parc_vals[union[p]] = zList[p]

rh_masked, lh_masked = parc_list_to_surf(parc_vals, 500)

fn = base_dir+'figures/GLMER_grp-all_'+task+'_predicting_quiz_z195'
four_panel_surfplot(rh_masked, lh_masked,fn,cmap_method='custom',custom_vmax=3,custom_vmin=-3,colormap='coolwarm',
                   title='Quiz Score predicting Item Neural Score -- Permuted Z>1.95')

fn = base_dir+'figures/GLMER_grp-all_'+task+'_predicting_quiz_z195_dorsal_ventral'
dors_vent_surfplot(rh_masked, lh_masked,fn,cmap_method='custom',custom_vmax=3,custom_vmin=-3,colormap='coolwarm',bg_on_data=True,
                   title='Quiz Score predicting Item Neural Score -- Permuted Z>1.95')